In [2]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import sys
import re
from glob import glob
import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
#  sys.path.append(f"/mnt/c/Git/go/kaggle/github/library/")
import utils
from utils import logger_func, mkdir_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

import eda
from utils import get_categorical_features, get_numeric_features, parallel_process
from feature_engineering import diff_feature, division_feature, product_feature, cnt_encoding, select_category_value_agg, exclude_feature, target_encoding
from tqdm import tqdm

#========================================================================
# Global Variable
sys.path.append(f"../py")
from info_home_credit import hcdr_key_cols
key, target, ignore_list = hcdr_key_cols()
#========================================================================

base = utils.read_df_pkl('../input/base0*').set_index(key)

100%|██████████| 3/3 [00:00<00:00, 204.59it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
"""
Bureau
"""
'''
#========================================================================
# BUREAU
CREDIT_ACTIVE
Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64
CREDIT_CURRENCY
currency 1    1715020
currency 2       1224
currency 3        174
currency 4         10
Name: CREDIT_CURRENCY, dtype: int64
CREDIT_TYPE
Consumer credit                                 1251615
Credit card                                      402195
Car loan                                          27690
Mortgage                                          18391
Microloan                                         12413
Loan for business development                      1975
Another type of loan                               1017
Unknown type of loan                                555
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     27
Loan for the purchase of equipment                   19
Loan for purchase of shares (margin lending)          4
Mobile operator loan                                  1
Interbank credit                                      1
Name: CREDIT_TYPE, dtype: int64
#======================================================================== 
'''
bur = utils.read_df_pkl(path='../input/clean_bur*.p')
display(bur.columns)

100%|██████████| 3/3 [00:01<00:00,  2.76it/s]


Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')

In [7]:
prefix = 'f001_bur_'
df = bur

org_col_list = df.columns.tolist()
num_list = get_numeric_features(df, ignore_list)

# df['AMT_CREDIT_SUM_per_AMT_ANNUITY'] = df['AMT_CREDIT_SUM'] / (df['AMT_ANNUITY'] + 1)

# df['AMT_CREDIT_SUM_diff_AMT_CREDIT_DEBT'] = df['AMT_CREDIT_SUM'] - df['AMT_CREDIT_SUM_DEBT']
# df['AMT_CREDIT_SUM_per_AMT_CREDIT_DEBT'] = df['AMT_CREDIT_SUM'] / (df['AMT_CREDIT_SUM_DEBT'] + 1)

# df['AMT_CREDIT_SUM_diff_AMT_CREDIT_DEBT_per_AMT_ANNUITY'] = df['AMT_CREDIT_SUM_diff_AMT_CREDIT_DEBT'] / (df['AMT_ANNUITY'] + 1)

# # overdue
# df['AMT_CREDIT_SUM_per_SUM_OVERDUE'] = df['AMT_CREDIT_SUM_OVERDUE'] / (df['AMT_CREDIT_SUM'] + 1)
# df['AMT_CREDIT_SUM_per_MAX_OVERDUE'] = df['AMT_CREDIT_MAX_OVERDUE'] / (df['AMT_CREDIT_SUM'] + 1)

# # limit
# df['AMT_CREDIT_SUM_per_LIMIT'] = df['AMT_CREDIT_SUM_LIMIT'] / (df['AMT_CREDIT_SUM'] + 1)
# df['AMT_CREDIT_SUM_diff_LIMIT'] = df['AMT_CREDIT_SUM_LIMIT'] - df['AMT_CREDIT_SUM']

# # term
# df['PLAN_TERM'] = df['DAYS_CREDIT_ENDDATE'] - df['DAYS_CREDIT']
# df['FACT_TERM'] = df['DAYS_ENDDATE_FACT'] - df['DAYS_CREDIT']
# df['UPDATE_TERM'] = df['DAYS_CREDIT_UPDATE'] - df['DAYS_CREDIT']

# # per term
# df['AMT_CREDIT_SUM_per_FACT_TERM'] = df['AMT_CREDIT_SUM'] / (df['FACT_TERM'] + 1)
# df['AMT_CREDIT_SUM_per_PLAN_TERM'] = df['AMT_CREDIT_SUM'] / (df['PLAN_TERM'] + 1)

# num_list = get_numeric_features(df=df, ignore_list=ignore_list)
# cat_list = get_categorical_features(df=df, ignore_list=[])

aggs = {}

# raw feature version
for col in num_list:
    aggs[col] = ['sum', 'mean', 'max', 'min', 'std']

# for col in df.columns:
#     if col in org_col_list:
#         continue
#     aggs[col] = ['sum', 'mean', 'max', 'min', 'std']
    
    
df_agg = df.groupby(key).agg(aggs)
new_col_list = [f"{col}-{method}" for col in aggs.keys() for method in aggs[col]]
df_agg.columns = new_col_list

print(df_agg.shape)
display(df_agg.head())

df_agg = base.join(df_agg)
utils.save_feature(df_feat=df_agg, is_train=2, target=target, ignore_list=[key, target], prefix=prefix)

(305773, 60)


,DAYS_CREDIT-sum,DAYS_CREDIT-mean,DAYS_CREDIT-max,DAYS_CREDIT-min,DAYS_CREDIT-std,CREDIT_DAY_OVERDUE-sum,CREDIT_DAY_OVERDUE-mean,CREDIT_DAY_OVERDUE-max,CREDIT_DAY_OVERDUE-min,CREDIT_DAY_OVERDUE-std,DAYS_CREDIT_ENDDATE-sum,DAYS_CREDIT_ENDDATE-mean,DAYS_CREDIT_ENDDATE-max,DAYS_CREDIT_ENDDATE-min,DAYS_CREDIT_ENDDATE-std,DAYS_ENDDATE_FACT-sum,DAYS_ENDDATE_FACT-mean,DAYS_ENDDATE_FACT-max,DAYS_ENDDATE_FACT-min,DAYS_ENDDATE_FACT-std,AMT_CREDIT_MAX_OVERDUE-sum,AMT_CREDIT_MAX_OVERDUE-mean,AMT_CREDIT_MAX_OVERDUE-max,AMT_CREDIT_MAX_OVERDUE-min,AMT_CREDIT_MAX_OVERDUE-std,CNT_CREDIT_PROLONG-sum,CNT_CREDIT_PROLONG-mean,CNT_CREDIT_PROLONG-max,CNT_CREDIT_PROLONG-min,CNT_CREDIT_PROLONG-std,AMT_CREDIT_SUM-sum,AMT_CREDIT_SUM-mean,AMT_CREDIT_SUM-max,AMT_CREDIT_SUM-min,AMT_CREDIT_SUM-std,AMT_CREDIT_SUM_DEBT-sum,AMT_CREDIT_SUM_DEBT-mean,AMT_CREDIT_SUM_DEBT-max,AMT_CREDIT_SUM_DEBT-min,AMT_CREDIT_SUM_DEBT-std,AMT_CREDIT_SUM_LIMIT-sum,AMT_CREDIT_SUM_LIMIT-mean,AMT_CREDIT_SUM_LIMIT-max,AMT_CREDIT_SUM_LIMIT-min,AMT_CREDIT_SUM_LIMIT-std,AMT_CREDIT_SUM_OVERDUE-sum,AMT_CREDIT_SUM_OVERDUE-mean,AMT_CREDIT_SUM_OVERDUE-max,AMT_CREDIT_SUM_OVERDUE-min,AMT_CREDIT_SUM_OVERDUE-std,DAYS_CREDIT_UPDATE-sum,DAYS_CREDIT_UPDATE-mean,DAYS_CREDIT_UPDATE-max,DAYS_CREDIT_UPDATE-min,DAYS_CREDIT_UPDATE-std,AMT_ANNUITY-sum,AMT_ANNUITY-mean,AMT_ANNUITY-max,AMT_ANNUITY-min,AMT_ANNUITY-std
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-5145,-735.000000,-49,-1572,489.942514,0,0.0,0,0,0.0,577.0,82.428571,1778.0,-1329.0,1032.859277,-3302.0,-825.500000,-544.0,-1328.0,369.078582,0.000,NaN,NaN,NaN,NaN,0,0.0,0,0,0.0,1453365.000,207623.571429,378000.0,85500.0,122544.544510,596686.5,85240.928571,373239.0,0.0,137485.631124,0.000,0.00000,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,-652.0,-93.142857,-6.0,-155.0,77.204120,24817.5,3545.357143,10822.5,0.0,4800.607529
100002,-6992,-874.000000,-103,-1437,431.451040,0,0.0,0,0,0.0,-2094.0,-349.000000,780.0,-1072.0,767.490977,-4185.0,-697.500000,-36.0,-1185.0,515.992539,8405.145,1681.029,5043.645,0.0,2363.2469,0,0.0,0,0,0.0,865055.565,108131.945625,450000.0,0.0,146075.557435,245781.0,49156.200000,245781.0,0.0,109916.604716,31988.565,7997.14125,31988.565,0.0,15994.2825,0.0,0.0,0.0,0.0,0.0,-3999.0,-499.875000,-7.0,-1185.0,518.522472,0.0,0.000000,0.0,0.0,0.000000
100003,-5603,-1400.750000,-606,-2586,909.826128,0,0.0,0,0,0.0,-2178.0,-544.500000,1216.0,-2434.0,1492.770467,-3292.0,-1097.333333,-540.0,-2131.0,896.097279,0.000,0.000,0.000,0.0,0.0000,0,0.0,0,0,0.0,1017400.500,254350.125000,810000.0,22248.0,372269.465535,0.0,0.000000,0.0,0.0,0.000000,810000.000,202500.00000,810000.000,0.0,405000.0000,0.0,0.0,0.0,0.0,0.0,-3264.0,-816.000000,-43.0,-2131.0,908.053963,0.0,NaN,NaN,NaN,NaN
100004,-1734,-867.000000,-408,-1326,649.124025,0,0.0,0,0,0.0,-977.0,-488.500000,-382.0,-595.0,150.613744,-1065.0,-532.500000,-382.0,-683.0,212.839141,0.000,0.000,0.000,0.0,NaN,0,0.0,0,0,0.0,189037.800,94518.900000,94537.8,94500.0,26.728636,0.0,0.000000,0.0,0.0,0.000000,0.000,0.00000,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,-1064.0,-532.000000,-382.0,-682.0,212.132034,0.0,NaN,NaN,NaN,NaN
100005,-572,-190.666667,-62,-373,162.297053,0,0.0,0,0,0.0,1318.0,439.333333,1324.0,-128.0,776.274007,-123.0,-123.000000,-123.0,-123.0,NaN,0.000,0.000,0.000,0.0,NaN,0,0.0,0,0,0.0,657126.000,219042.000000,568800.0,29826.0,303238.426806,568408.5,189469.500000,543087.0,0.0,306503.339003,0.000,0.00000,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,-163.0,-54.333333,-11.0,-121.0,58.594653,4261.5,1420.500000,4261.5,0.0,2460.378172


(307511,) | DAYS_CREDIT-sum
(307511,) | DAYS_CREDIT-mean
(307511,) | DAYS_CREDIT-max
(307511,) | DAYS_CREDIT-min
(307511,) | DAYS_CREDIT-std
(307511,) | CREDIT_DAY_OVERDUE-sum
(307511,) | CREDIT_DAY_OVERDUE-mean
(307511,) | CREDIT_DAY_OVERDUE-max
(307511,) | CREDIT_DAY_OVERDUE-min
(307511,) | CREDIT_DAY_OVERDUE-std
(307511,) | DAYS_CREDIT_ENDDATE-sum
(307511,) | DAYS_CREDIT_ENDDATE-mean
(307511,) | DAYS_CREDIT_ENDDATE-max
(307511,) | DAYS_CREDIT_ENDDATE-min
(307511,) | DAYS_CREDIT_ENDDATE-std
(307511,) | DAYS_ENDDATE_FACT-sum
(307511,) | DAYS_ENDDATE_FACT-mean
(307511,) | DAYS_ENDDATE_FACT-max
(307511,) | DAYS_ENDDATE_FACT-min
(307511,) | DAYS_ENDDATE_FACT-std
(307511,) | AMT_CREDIT_MAX_OVERDUE-sum
(307511,) | AMT_CREDIT_MAX_OVERDUE-mean
(307511,) | AMT_CREDIT_MAX_OVERDUE-max
(307511,) | AMT_CREDIT_MAX_OVERDUE-min
(307511,) | AMT_CREDIT_MAX_OVERDUE-std
(307511,) | CNT_CREDIT_PROLONG-sum
(307511,) | CNT_CREDIT_PROLONG-mean
(307511,) | CNT_CREDIT_PROLONG-max
(307511,) | CNT_CREDIT_PROLON

In [4]:
from new_FE import CategoricalEncoding

#========================================================================
# Feature Engineering 
df = bur
cat_enc = CategoricalEncoding(data=df, key=key, target=target, ignore_list=[key, target], base=base)
prefix = 'f001_bur_'
cat_list = get_categorical_features(df, ignore_list)
df_ohe = cat_enc.go_ohe(cat_list=cat_list, method_list=['sum', 'mean'], prefix=prefix)
df_label = cat_enc.go_label(cat_list=cat_list, method_list=['mean', 'std', 'max', 'min'], prefix=prefix)
#========================================================================

100%|██████████| 3/3 [00:01<00:00,  1.53it/s]


(307511,) | ohe_CREDIT_ACTIVE_Active-sum
(307511,) | ohe_CREDIT_ACTIVE_Active-mean
(307511,) | ohe_CREDIT_ACTIVE_Bad debt-sum
(307511,) | ohe_CREDIT_ACTIVE_Bad debt-mean
(307511,) | ohe_CREDIT_ACTIVE_Closed-sum
(307511,) | ohe_CREDIT_ACTIVE_Closed-mean
(307511,) | ohe_CREDIT_ACTIVE_Sold-sum
(307511,) | ohe_CREDIT_ACTIVE_Sold-mean
(307511,) | ohe_CREDIT_CURRENCY_currency 1-sum
(307511,) | ohe_CREDIT_CURRENCY_currency 1-mean
(307511,) | ohe_CREDIT_TYPE_Another type of loan-sum
(307511,) | ohe_CREDIT_TYPE_Another type of loan-mean
(307511,) | ohe_CREDIT_TYPE_Car loan-sum
(307511,) | ohe_CREDIT_TYPE_Car loan-mean
(307511,) | ohe_CREDIT_TYPE_Cash loan _non-earmarked_-sum
(307511,) | ohe_CREDIT_TYPE_Cash loan _non-earmarked_-mean
(307511,) | ohe_CREDIT_TYPE_Consumer credit-sum
(307511,) | ohe_CREDIT_TYPE_Consumer credit-mean
(307511,) | ohe_CREDIT_TYPE_Credit card-sum
(307511,) | ohe_CREDIT_TYPE_Credit card-mean
(307511,) | ohe_CREDIT_TYPE_Interbank credit-sum
(307511,) | ohe_CREDIT_TYPE_Int

100%|██████████| 3/3 [00:00<00:00, 16.15it/s]

[One Hot Encoding] done in 5 s
[Factorize Encoding] done in 0 s


[Aggregation] done in 1 s
(307511,) | label_CREDIT_ACTIVE-mean
(307511,) | label_CREDIT_ACTIVE-std
(307511,) | label_CREDIT_ACTIVE-max
(307511,) | label_CREDIT_ACTIVE-min
(307511,) | label_CREDIT_CURRENCY-mean
(307511,) | label_CREDIT_CURRENCY-std
(307511,) | label_CREDIT_CURRENCY-max
(307511,) | label_CREDIT_CURRENCY-min
(307511,) | label_CREDIT_TYPE-mean
(307511,) | label_CREDIT_TYPE-std
(307511,) | label_CREDIT_TYPE-max
(307511,) | label_CREDIT_TYPE-min
(48744,) | label_CREDIT_ACTIVE-mean
(48744,) | label_CREDIT_ACTIVE-std
(48744,) | label_CREDIT_ACTIVE-max
(48744,) | label_CREDIT_ACTIVE-min
(48744,) | label_CREDIT_CURRENCY-mean
(48744,) | label_CREDIT_CURRENCY-std
(48744,) | label_CREDIT_CURRENCY-max
(48744,) | label_CREDIT_CURRENCY-min
(48744,) | label_CREDIT_TYPE-mean
(48744,) | label_CREDIT_TYPE-std
(48744,) | label_CREDIT_TYPE-max
(48744,) | label_CREDIT_TYPE-min
